# <center> Home work for model 6

In [18]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression

### Загрузим данные и проверим их читаемость, предобработка данных

In [19]:
data = pd.read_excel('../data/data_ford_price.xlsx') 
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [20]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [21]:
estimator = LinearRegression() # модель обучения
selector = RFE(estimator, n_features_to_select=3, step=1)
# метод выбора прищнаков , где 
# n_features_to_select - количество итоговых признаков
# step =  количество итераций
selector = selector.fit(X_train, y_train) # обучаем модель
 
selector.get_feature_names_out() # выдает итог выбранных признаков

array(['year', 'cylinders', 'lat'], dtype=object)

In [22]:
y_predicted = selector.predict(X_test)
 
mae_rfe = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 5096.570


In [23]:
# создадим эстиматор для SelectKBest
selecor_2 = SelectKBest(f_regression, k=3) # первым передается статистический метод, и количество нужных признаков
X_new = selecor_2.fit_transform(X_train, y_train)


print(selecor_2.get_feature_names_out())

model = estimator.fit(X_new, y_train)
y_predicted_2 = model.predict(selecor_2.transform(X_test)) # в содель передаем трансформированные тестовые данные
mae_SK = mean_absolute_error(y_test, y_predicted_2)
print('MAE: %.3f' % mae_SK)

['year' 'cylinders' 'odometer']
MAE: 4708.946


На вопрос "Какой метод отбора признаков показал наилучший результат на тестовой выборке?" из полученных результатов можно сделать слудющий вывод, что лучший результат на данной выборке показал **RFE**  так как метрика  *MAE*  у него примерно 4% выше, что может сыграть значительную роль. Отбор *SelectKBest* отработал хуже, так как он выбрал одной из луших фичей odometr, что на самом деле таковым не являтся.